In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

```markdown
# 🍜 AI Food Recommender: What to Eat & Where to Have It

## 🧠 Project Objective

The objective of this project is to develop an **AI agent** that can recommend **what to eat** and **where to have it** based on:

- User preferences
- Dietary restrictions
- Time of day (e.g., breakfast, lunch, dinner)
- Current weather condition
- User's current location and destination

This system will help users discover new **restaurants, hawkers, or food locations**, and make more personalized and informed decisions about their meals.

---

## 📌 Project Components

### 1. User Input Module
Collects the following:
- Preferred taste/flavor (e.g., spicy, savory, sweet)
- Dietary restrictions (e.g., vegetarian, halal, no peanuts)
- Time of day (breakfast, lunch, dinner)
- Current location (e.g., Bangsar)
- Destination (optional, e.g., heading to Sunway)
- Food type preference (optional: street food, café, healthy, quick bites)

---

### 2. Context-Aware Fetchers
- **Weather API** (stub or simulated): Fetch current weather to influence food types (e.g., soup on rainy days).
- **Time logic**: Suggest meals appropriate for breakfast, lunch, or dinner.
- **Place discovery module**:
  - Simulated or real API (e.g., Google Maps, Foursquare)
  - Suggest restaurants/hawkers near the current location or destination

---

### 3. LLM-Based Recommendation Engine
- Uses prompt-based logic to generate recommendations from:
  - User inputs
  - Current weather
  - Time of day
  - Nearby food options (filtered list)
- Outputs:
  - **What to eat** (dish suggestion)
  - **Where to eat** (restaurant/hawker name + location)
  - **Explanation** (why this choice was made)

---

### 4. Simple Web UI (via Streamlit)
- Web form to input preferences
- Button to generate recommendation
- Output area displaying:
  - Dish recommendation
  - Restaurant/hawker details (simulated or real)
  - Map (optional or stub)
  - Reasoning from LLM

---

## 🗂️ Suggested Project Structure (Kaggle-Compatible)

```plaintext
/AI-Food-Recommender
│
├── recommender_agent.ipynb         # Main notebook for simulation
│
├── data/
│   ├── food_places_sample.csv      # Sample food places (manual or API-based)
│   └── food_tags.csv               # Dishes with tags (diet, cuisine, etc.)
│
├── utils/
│   ├── weather.py                  # Simulate or fetch weather
│   ├── food_places.py              # Place lookup logic (API or static CSV)
│   └── llm_agent.py                # Prompt template and response parsing
│
├── streamlit_app.py (optional)     # Streamlit app for local use
└── README.md
```

---

## 📦 Dependencies (for Kaggle Notebook)

You can install external packages if needed (only if not already available on Kaggle):

```python
!pip install openai streamlit geopy requests
```

---

## 🧪 Example User Scenario

**Input**:
- Preferences: Spicy, Halal
- Time: Dinner
- Location: Petaling Jaya
- Destination: Subang Jaya
- Weather: Rainy

**AI Recommendation Output**:
- **🍛 Dish**: Nasi Kukus Ayam Berempah
- **📍 Place**: Warong Chef Mama, SS15 Subang
- **💬 Reason**: "Given the rainy evening and your preference for spicy halal food, Nasi Kukus is a warm, satisfying option. Warong Chef Mama is on your way and highly rated for this dish."

---

## ✅ Development Phases

| Phase | Task |
|-------|------|
| 1 | Create user input form in notebook |
| 2 | Mock weather + location data modules |
| 3 | Build static food place CSVs (mock Foursquare/Maps API) |
| 4 | Integrate LLM prompt system |
| 5 | Simulate and display results |
| 6 | (Optional) Streamlit version for local testing/demo |

---

## 📘 Notes

- Kaggle doesn’t support Streamlit inside the notebook, but you can simulate all logic using standard input/output.
- You may simulate location/weather APIs using preloaded JSON or CSV files.
- For LLMs, use OpenAI API, HuggingFace, or simulate outputs if API access is limited.

---

## 🧩 Phase 1: User Input Form (Simulated in Notebook)

This section captures user preferences and context needed for the AI food recommender. Since this is a Kaggle notebook, we'll simulate user input using Python variables (not interactive widgets like Streamlit).

The input fields include:
- Preferred flavor/taste (e.g., spicy, sweet)
- Dietary restrictions (e.g., vegetarian, halal)
- Time of day (e.g., breakfast, lunch, dinner)
- Current location (e.g., Bangsar, KL Sentral)
- Destination (optional, e.g., heading to Pavilion KL)
- Food type preference (optional, e.g., street food, café)


In [2]:
# Simulated user input in Kaggle notebook

user_input = {
    "preferred_flavor": "spicy",             # e.g., spicy, sweet, savory, sour
    "dietary_restrictions": ["halal"],       # e.g., vegetarian, halal, gluten-free
    "time_of_day": "lunch",                  # breakfast, lunch, dinner
    "current_location": "KL Sentral",        # starting point
    "destination": "Pavilion KL",            # optional
    "food_type": "street food"               # optional, e.g., street food, healthy, cafe
}

# Print to confirm input
print("✅ User Input Collected:")
for key, value in user_input.items():
    print(f"{key.replace('_', ' ').capitalize()}: {value}")


✅ User Input Collected:
Preferred flavor: spicy
Dietary restrictions: ['halal']
Time of day: lunch
Current location: KL Sentral
Destination: Pavilion KL
Food type: street food


## 🌤️ Phase 2: Get Current Weather Report Based on Location

In this phase, we fetch the current weather using the user's current location. This adds real-time context to the food recommendation (e.g., suggesting hot soup during rainy weather).

Steps:
1. Convert the user's location (e.g., "KL Sentral") into geographic coordinates using Geocoding.
2. Use the coordinates to fetch the current weather from OpenWeatherMap API.


## 🌤️ Fetch Weather Using Secure API Key (Kaggle Secrets)

This phase fetches real-time weather conditions using OpenWeatherMap API. To protect sensitive credentials, we use **Kaggle Secrets** instead of hardcoding the API key.


In [3]:
import requests
from geopy.geocoders import Nominatim
from kaggle_secrets import UserSecretsClient
import os

def fetch_weather(user_input):
    """
    Fetches weather data for the user's current location.
    """
    # Step 1: Geocode current location
    location_name = user_input["current_location"]
    geolocator = Nominatim(user_agent="food-recommender")
    
    print(f"🔍 Looking up coordinates for: {location_name}")
    location = geolocator.geocode(location_name)

    if location:
        lat, lon = location.latitude, location.longitude
        print(f"📍 Coordinates for {location_name}: ({lat}, {lon})")

        # Step 2: Access OpenWeather API key from Kaggle secrets (using environment variable)
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("OPENWEATHER_API_KEY")
        
        if not api_key:
            raise ValueError("❌ API key not found. Ensure you've added the secret to your notebook settings.")
        
        print("🔑 API key retrieved successfully.")

        # Step 3: Call OpenWeatherMap API
        weather_url = (
            f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
        )
        
        print("🌐 Fetching weather data from OpenWeatherMap...")
        response = requests.get(weather_url)

        if response.status_code == 200:
            weather_data = response.json()
            weather_desc = weather_data["weather"][0]["description"]
            temperature = weather_data["main"]["temp"]
            print(f"🌤️ Current weather at {location_name}: {weather_desc}, {temperature}°C")
        else:
            print(f"⚠️ Failed to fetch weather data. Response code: {response.status_code}")
    else:
        print(f"⚠️ Could not find coordinates for location: {location_name}")


if __name__ == "__main__":
    # Example user input
    user_input = {
        "current_location": "Kuala Lumpur"
    }

    # Fetch weather information
    fetch_weather(user_input)

🔍 Looking up coordinates for: Kuala Lumpur
📍 Coordinates for Kuala Lumpur: (3.1526589, 101.7022205)
🔑 API key retrieved successfully.
🌐 Fetching weather data from OpenWeatherMap...
🌤️ Current weather at Kuala Lumpur: few clouds, 32.66°C


## 📍 Phase 3: Dynamic Food Place Recommendation using Google Places API

In this phase, we expand the user input form to collect dietary preferences, food types, and travel context. Then, we connect to the **Google Places API** to dynamically suggest food venues near the user’s *destination* based on these criteria.

This hybrid approach simulates how an AI agent can generate contextualized and culturally relevant food recommendations — a critical step toward building a real-time decision support system for Malaysian and Asian cuisine lovers.

### 👣 Steps
1. Collect user preferences and travel context (e.g. location, destination).
2. Use Google Places API to search for food venues near the destination.
3. Display the top matched venues in a table.


In [4]:
# 📥 User Input Form
import ipywidgets as widgets
from IPython.display import display

# Define input widgets
dietary_input = widgets.Dropdown(
    options=["halal", "vegetarian", "vegan", "no preference"],
    description="Dietary:"
)

food_type_input = widgets.Text(
    value="street food",
    description="Food Type:"
)

current_location_input = widgets.Text(
    value="KL Sentral",
    description="Current:"
)

destination_input = widgets.Text(
    value="Pavilion KL",
    description="Destination:"
)

submit_button = widgets.Button(description="🔍 Get Recommendations")

# Display the widgets
display(dietary_input, food_type_input, current_location_input, destination_input, submit_button)

# Store input when user clicks button
user_input = {}

def on_submit_clicked(b):
    user_input["dietary"] = dietary_input.value
    user_input["food_type"] = food_type_input.value
    user_input["current_location"] = current_location_input.value
    user_input["destination"] = destination_input.value
    print("✅ Input received:", user_input)

submit_button.on_click(on_submit_clicked)


Dropdown(description='Dietary:', options=('halal', 'vegetarian', 'vegan', 'no preference'), value='halal')

Text(value='street food', description='Food Type:')

Text(value='KL Sentral', description='Current:')

Text(value='Pavilion KL', description='Destination:')

Button(description='🔍 Get Recommendations', style=ButtonStyle())

In [7]:
import requests
from kaggle_secrets import UserSecretsClient
import pandas as pd

def search_google_places(user_input, radius=3000):
    """
    Query Google Places API based on user input for food recommendations.
    """
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_PLACES_API_KEY")
    
    query = f"{user_input['dietary']} {user_input['food_type']} food"
    location = user_input['destination']

    print(f"📡 Searching for: '{query}' near '{location}'")

    url = (
        "https://maps.googleapis.com/maps/api/place/textsearch/json"
        f"?query={query}+restaurant+in+{location}"
        f"&radius={radius}&key={api_key}"
    )

    response = requests.get(url)
    results = []

    if response.status_code == 200:
        data = response.json()
        for place in data.get("results", []):
            results.append({
                "name": place.get("name"),
                "address": place.get("formatted_address"),
                "rating": place.get("rating"),
                "total_reviews": place.get("user_ratings_total"),
                "types": ", ".join(place.get("types", []))
            })
    else:
        print(f"❌ Google Places API Error: {response.status_code}")
    
    return pd.DataFrame(results)


In [8]:
# ⚠️ Make sure user_input is filled by submitting the form first
if user_input:
    recommended_places = search_google_places(user_input)
    if not recommended_places.empty:
        display(recommended_places.head(5))
    else:
        print("😕 No suitable places found.")
else:
    print("⚠️ Please fill and submit the user input form first.")


📡 Searching for: 'halal street food food' near 'Pavilion KL'


,name,address,rating,total_reviews,types
0,Ei8ht Avenue @ Pavilion KL,"Lot 1.29.01 - Lot 1.29.04, Pavilion KL, 168, J...",3.4,187,"food, point_of_interest, establishment"
1,Food Republic at Pavilion KL,"Lot 1.41.00 - 1.51.00 & P1.13.00 - P1.20.00, 1...",4.2,3154,"food, point_of_interest, establishment"
2,ParaThai @ Pavilion KL,"Lot 1.03 & 1.04, Level 1, Pavilion KL, 168, Jl...",4.8,2329,"restaurant, food, point_of_interest, establish..."
3,Mee Tarik @ Food Republic,"Pavilion, Level 1, 168, Jln Bukit Bintang, Buk...",4.5,40,"food, point_of_interest, establishment"
4,Serai@Pavilion,"Lot 7 . 01 . 04, 168, Jln Bukit Bintang, Bukit...",4.7,3904,"restaurant, food, point_of_interest, establish..."
